# Azure AI Foundry o3-mini examples

<img src="o3.jpg" width=500>

Azure OpenAI o-series models are designed to tackle reasoning and problem-solving tasks with increased focus and capability. These models spend more time processing and understanding the user's request, making them exceptionally strong in areas like science, coding, and math compared to previous iterations.

### Key Capabilities of these models
- o1 added advanced image analysis capabilities with the new version. Enhance your prompts and context with images for additional insights.
- o3-mini follows o1 mini but adds the features supported by o1 like function calling and tools.
- Complex Code Generation: Capable of generating algorithms and handling advanced coding tasks to support developers.
- Advanced Problem Solving: Ideal for comprehensive brainstorming sessions and addressing multifaceted challenges.
- Complex Document Comparison: Perfect for analyzing contracts, case files, or legal documents to identify subtle differences.
- Instruction Following and Workflow Management: Particularly effective for managing workflows requiring shorter contexts.

### Features and properties supported in o3-mini model
- Supports both System message and the new Developer message to improve upgrade experience.
- Reasoning effort as in high, medium, and low. It controls whether the model thinks "less" or "more" in terms of applying cognitive reasoning.
- Structured outputs and functions/tools.
- Context window: 200K, Max Completion Tokens: 100K

### Model Variants
- o3-mini: Now includes the o1 features with significant cost-efficiencies for scenarios requiring high performance.
- o1: The most capable model in the o1 series, offering enhanced reasoning abilities. Now generally available.
- o1-mini: A faster and more cost-efficient option in the o1 series, ideal for coding tasks requiring speed and lower resource consumption.

### Documentation
- https://azure.microsoft.com/en-us/blog/announcing-the-availability-of-the-o3-mini-reasoning-model-in-microsoft-azure-openai-service/
- https://techcommunity.microsoft.com/discussions/marketplace-forum/o3-mini-reasoning-model-now-available-in-microsoft-azure-openai-service/4372800
- https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python-secure

In [1]:
import gradio as gr
import openai
import os
import sys
import tempfile
import time

from dotenv import load_dotenv
from datetime import datetime
from openai import AzureOpenAI
from IPython.display import display, Markdown

In [2]:
print(f"Python version: {sys.version}")
print(f"OpenAI version: {openai.__version__}")

Python version: 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]
OpenAI version: 1.61.1


In [3]:
print('OK') if load_dotenv("azure.env") else print('ERROR: Check file location or name.')

OK


In [4]:
print(datetime.now())

2025-03-13 16:28:17.955679


## Settings

In [5]:
model = "o3-mini"  # As deployed in Azure AI Foundry

api_version = "2025-01-01-preview"  # Subject to change. Check: https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python-secure#api--feature-support

In [6]:
print(f"We will use {model} from Azure AI Foundry")

We will use o3-mini from Azure AI Foundry


In [55]:
def o3mini(prompt: str, reasoning_effort="medium"):
    """
    Sends a chat completion request to the Azure OpenAI API o3 mini model.
    Args:
        prompt (str): The input prompt to generate a response for.
    Returns:
        response (openai.Completion): The response object containing the generated completion.
    """
    try:
        # Azure OpenAI client
        start = time.time()

        aoai_client = AzureOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=api_version,
        )
        # Calling the model
        response = aoai_client.chat.completions.create(
            model=model,
            messages=[{
                "role": "user",
                "content": prompt
            }],
            max_completion_tokens=100_000,
            reasoning_effort=reasoning_effort,  # set to low, medium or high
        )

        elapsed = time.time() - start
        # Cost computing
        cost_input_token = 1.0470 / 1_000_000
        cost_output_token = 4.187884 / 1_000_000
        total_cost = (response.usage.prompt_tokens * cost_input_token) + (response.usage.completion_tokens * cost_output_token)
        # Elapsed time
        elapsed_time_msg = f"Completed in {time.strftime('%H:%M:%S' + str(elapsed % 1)[2:2], time.gmtime(elapsed))}"
        # Message generation
        msg_to_display = f"Prompt Tokens = {response.usage.prompt_tokens} | Completion tokens = {response.usage.completion_tokens} | Total tokens = {response.usage.total_tokens}\nTotal cost = {total_cost} EUR\n{elapsed_time_msg}\n"
        print(msg_to_display)

        return response

    except Exception as e:
        print(f"[ERROR] {e}")
        return None

## Quick test

In [56]:
prompt = "Who won the 2022 FIFA World Cup? Consider different perspectives, analyze the impact of the victory on global football, and discuss the team’s tactical approach in key matches."

resp = o3mini(prompt, reasoning_effort="low")

Prompt Tokens = 41 | Completion tokens = 620 | Total tokens = 661
Total cost = 0.00263941508 EUR
Completed in 00:00:13



In [57]:
print(resp.model_dump_json(indent=5))

{
     "id": "chatcmpl-BAg4W8WOLzeXr8JoS6pZbGgmb3nIr",
     "choices": [
          {
               "finish_reason": "stop",
               "index": 0,
               "logprobs": null,
               "message": {
                    "content": "Argentina won the 2022 FIFA World Cup. Their triumph was not only a significant sporting achievement but also a moment of deep cultural and emotional impact, both in Argentina and across the world of football.\n\n1. Global and Cultural Impact:\n • For Argentina, the victory was a historic milestone, cementing the legacy of the team and its talismanic captain Lionel Messi. It fulfilled long-held national dreams and united a country that has faced economic and social challenges, offering a moment of collective pride and celebration.\n • Internationally, Argentina’s win was seen as a triumph of skill, teamwork, and sheer determination. It reinvigorated interest in South American football and showcased the region's ability to compete at the highest 

In [58]:
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Argentina won the 2022 FIFA World Cup. Their triumph was not only a significant sporting achievement but also a moment of deep cultural and emotional impact, both in Argentina and across the world of football.

1. Global and Cultural Impact:
 • For Argentina, the victory was a historic milestone, cementing the legacy of the team and its talismanic captain Lionel Messi. It fulfilled long-held national dreams and united a country that has faced economic and social challenges, offering a moment of collective pride and celebration.
 • Internationally, Argentina’s win was seen as a triumph of skill, teamwork, and sheer determination. It reinvigorated interest in South American football and showcased the region's ability to compete at the highest levels alongside traditionally dominant European teams.
 • The win also inspired discussions about the evolving nature of global football. Many analysts pointed to the blend of experienced players and emerging talent as a sign that football excellen

In [59]:
resp.model

'o3-mini-2025-01-31'

## Basic examples

In [11]:
prompt = """
Imagine we are trying to figure out the employee hierarchy chart for the CONTOSO company with this:

Alan is the CEO of the company. Jane is the VP of Sales of EMEA. John is the global head of product development. Eric manages the shopping cart experience. Gilles manages sales in France. Sophie is responsible for sales for Italy. Marc reports to Sophie as well. Christine is in charge of user experience. Vincent is responsible of sales for Asia.
"""

resp = o3mini(prompt, reasoning_effort="low")
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 98 | Completion tokens = 676 | Total tokens = 774
Total cost = 0.0029336155840000004 EUR
Completed in 00:00:07

Based on the information provided, one reasonable interpretation of CONTOSO’s employee hierarchy could be as follows:

• Alan (CEO)  
 ○ Jane (VP of Sales, EMEA)  
  ‣ Gilles (Sales Manager for France)  
  ‣ Sophie (Sales Manager for Italy)  
   ▪ Marc (reports to Sophie)  
 ○ John (Global Head of Product Development)  
  ‣ Eric (Manager, Shopping Cart Experience)  
  ‣ Christine (in charge of User Experience)  
 ○ Vincent (Sales for Asia)

A few points to note about this assumed chart:
 – The hierarchy is based on the logical grouping of roles: Sales (with separate regional managers under a VP for EMEA), Product Development (with sub-teams for shopping cart experience and user experience), and an additional Sales function focused on Asia.
 – It’s assumed that Vincent, who is responsible for sales in Asia, reports directly to the CEO (or is positioned at a lev

In [12]:
prompt = "What are the differences between o1, o1-mini and o3-mini genAI models in terms of size and complexity, performance, usecases and architecture?"

resp = o3mini(prompt, reasoning_effort="medium")
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 38 | Completion tokens = 2074 | Total tokens = 2112
Total cost = 0.008725457416 EUR
Completed in 00:00:32

Below is an overview that contrasts the three variants—o1, o1-mini, and o3-mini—in key areas. (Keep in mind that while the exact details can vary based on implementation and release notes, the following is a general guide based on typical design decisions when creating “mini” variants of generative AI models.)

──────────────────────────────
1. Size and Complexity

• o1:  
 – Represents the full‑scale model with a high number of parameters.  
 – Has a deep, richly layered transformer architecture and large hidden dimensions.  
 – Offers the highest complexity, allowing it to capture very subtle patterns and context. 

• o1‑mini:  
 – A streamlined version of o1, created by substantially reducing the number of parameters and layers.  
 – Maintains much of the core transformer architecture of o1 but “downsized” for efficiency.  
 – Targets a reduction in resource req

## Developer role

In [13]:
aoai_client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=api_version,
)

resp = aoai_client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "developer",
            "content": "You are a helpful assistant."
        },
        {
            "role":
            "user",
            "content":
            "What steps should I think about when writing my first Python API?"
        },
    ],
    reasoning_effort='medium',  # low or medium or high
    max_completion_tokens=10000)

In [14]:
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

When building your first Python API, it’s helpful to break the process down into a number of steps. Here’s a guide you can use to structure your work:

1. Define Your Requirements and Goals  
   • Determine what functionality your API should provide.  
   • Identify your target users and how they will interact with the API.  
   • List the endpoints and operations (CRUD—create, read, update, delete) that the API should support.

2. Choose a Framework  
   • Evaluate frameworks such as Flask, FastAPI, and Django REST Framework.  
   • Flask is lightweight and good for learning and simple projects, while FastAPI offers fast performance and enhanced features like automatic OpenAPI documentation.  
   • Consider the pros and cons of each based on your project’s needs.

3. Set Up Your Development Environment  
   • Create a virtual environment using venv or conda to isolate dependencies.  
   • Use a package manager like pip to install the chosen framework and any other dependencies.  
   •

## Markdown export

In [15]:
resp = aoai_client.chat.completions.create(
    model=model,
    messages=[
        {
            "role":
            "developer",
            "content":
            "Formatting re-enabled - please enclose code blocks with appropriate markdown tags. You are a helpful assistant."
        },
        {
            "role":
            "user",
            "content":
            "What steps should I think about when writing my first Python API?"
        },
    ],
    reasoning_effort='low',  # low or medium or high
    max_completion_tokens=10000)

In [16]:
display(Markdown(resp.choices[0].message.content))

When writing your first Python API, it’s helpful to break down the process into clear, manageable steps. Here’s a list of considerations and steps to guide you:

1. **Define Goals and Requirements**
   - Identify what functionality your API should provide.
   - Determine who its users will be and what kind of data or operations they expect.
   - Outline the data models and business logic.

2. **Choose a Framework**
   - Explore popular frameworks like [Flask](https://flask.palletsprojects.com/), [Django REST framework](https://www.django-rest-framework.org/), or [FastAPI](https://fastapi.tiangolo.com/).
   - Consider ease of use, performance, community support, and built-in features like validation and documentation generation.

3. **Plan Out Your Endpoints**
   - Decide on the structure and naming conventions of your URL endpoints.
   - Choose HTTP methods (GET, POST, PUT, DELETE, etc.) for each endpoint.
   - Consider RESTful design principles to ensure a logical API structure.

4. **Design the Data Model and Persistence Layer**
   - Define the data structures or schemas your API will use.
   - Choose an appropriate database (SQL vs. NoSQL) and an ORM (if needed, e.g., SQLAlchemy).
   - Plan how your API will handle data access, updates, and error cases.

5. **Implement Authentication and Authorization**
   - Decide on the authentication methods (API keys, OAuth, JWT, etc.) based on your needs.
   - Set up user roles, permissions, or scopes to control access to resources.
   - Use libraries and middleware from your chosen framework to streamline the process.

6. **Handle Input Validation and Error Management**
   - Validate incoming request data using libraries or built-in framework tools.
   - Provide clear and consistent error messages; decide on your error response format.
   - Consider exception handling and logging to catch and diagnose issues effectively.

7. **Develop and Document Your API**
   - Write clean, well-commented code focusing on maintainability.
   - Use tools like Swagger/OpenAPI, Redoc, or built-in documentation tools (FastAPI, for instance, auto-generates docs) to document your endpoints.
   - Keep your documentation up-to-date with code changes.

8. **Test Your API**
   - Write unit tests and integration tests to validate your endpoints.
   - Use testing frameworks such as pytest and tools like Postman or curl for manual endpoint verification.
   - Consider automated testing for continuous integration setups.

9. **Consider Performance and Scalability**
   - Identify potential performance bottlenecks.
   - Think about caching strategies (e.g., using Redis) for frequently accessed data.
   - Choose a deployment strategy that allows horizontal scaling if necessary.

10. **Deploy**
    - Choose a hosting platform (e.g., Heroku, AWS, Google Cloud, DigitalOcean).
    - Set up a production-ready server with secure HTTPS configurations.
    - Use environment variables and configuration management to handle settings securely.

11. **Monitor and Iterate**
    - Implement logging and monitoring to track API usage and errors.
    - Gather feedback from early users or internal testing.
    - Improve your API based on performance data and user reports.

By following these steps, you can build a well-structured and maintainable Python API. Each step not only guides your development process but also helps ensure your API is secure, scalable, and user-friendly. Happy coding!

### Legal example

In [17]:
prompt = """
You are an expert lawyer. So am I.
We are working on a document that will allow Sophie Martin to purchase a property for $710,000.
The property is located at 1500 Diagonal Road, Contoso City, California.
The sellers are John and Jane Smith. Sophie Martin will pay $5,000 in earnest money, and wants the right to conduct an inspection.
The parties should close within 60 days. This sale is not contingent on financing.

Could you help me draft the document?
"""

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 110 | Completion tokens = 4273 | Total tokens = 4383
Total cost = 0.018009998332000003 EUR
Completed in 00:00:57

Below is a sample draft of a Real Estate Purchase Agreement for the transaction. (Note: This draft is provided for discussion purposes only and should be reviewed and customized to meet all legal requirements, including local California law, as well as the parties’ specific intentions. I recommend that both parties have counsel review and modify this document before execution.)

─────────────────────────────  
REAL ESTATE PURCHASE AGREEMENT

This Real Estate Purchase Agreement (this “Agreement”) is made and entered into as of __________________ (“Effective Date”) by and between John Smith and Jane Smith (collectively, the “Sellers”) and Sophie Martin (the “Buyer”). Sellers and Buyer are sometimes individually referred to as a “Party” and collectively as the “Parties.”

1. Property.  
  Sellers agree to sell and Buyer agrees to purchase that certain real prop

In [18]:
base_prompt = (
    "<prompt>You are a lawyer specializing in competition law, "
    "assisting business owners with their questions.</prompt>\n"
    "<policy>As a legal professional, provide clear and accurate "
    "information about competition law while maintaining "
    "confidentiality and professionalism. Avoid giving specific "
    "legal advice without sufficient context, and encourage clients "
    "to seek personalized counsel when necessary. Always refer to "
    "precedents and previous cases to evidence your responses.</policy>\n")

legal_query = (
    "<query>A larger company is offering suppliers incentives not to do "
    "business with me. Is this legal?</query>")

In [19]:
example_prompt = (
    "<prompt>You are a lawyer specializing in competition law, "
    "assisting business owners with their questions.</prompt>\n"
    "<policy>As a legal professional, provide clear and accurate "
    "information about competition law while maintaining "
    "confidentiality and professionalism. Avoid giving specific "
    "legal advice without sufficient context, and encourage clients "
    "to seek personalized counsel when necessary.</policy>\n"
    """<example>
<question>
I'm considering collaborating with a competitor on a joint marketing campaign. Are there any antitrust issues I should be aware of?
</question>
<response>
Collaborating with a competitor on a joint marketing campaign can raise antitrust concerns under U.S. antitrust laws, particularly the Sherman Antitrust Act of 1890 (15 U.S.C. §§ 1–7). Section 1 of the Sherman Act prohibits any contract, combination, or conspiracy that unreasonably restrains trade or commerce among the states.

**Key Considerations:**

1. **Per Se Illegal Agreements:** Certain collaborations are considered automatically illegal ("per se" violations), such as price-fixing, bid-rigging, and market allocation agreements. For example, in *United States v. Topco Associates, Inc.*, 405 U.S. 596 (1972), the Supreme Court held that market division agreements between competitors are per se illegal under the Sherman Act.

2. **Rule of Reason Analysis:** Collaborations that are not per se illegal are evaluated under the "rule of reason," which assesses whether the pro-competitive benefits outweigh the anti-competitive effects. In *Broadcast Music, Inc. v. Columbia Broadcasting System, Inc.*, 441 U.S. 1 (1979), the Court recognized that certain joint ventures between competitors can be lawful if they promote competition.

3. **Information Sharing Risks:** Sharing competitively sensitive information, such as pricing strategies or customer data, can lead to antitrust violations. The Department of Justice and the Federal Trade Commission caution against exchanges that could facilitate collusion (*Antitrust Guidelines for Collaborations Among Competitors*, 2000).

**Recommendations:**

- **Define the Scope:** Clearly delineate the parameters of the collaboration to focus on the marketing campaign without involving competitive aspects like pricing or market division.
- **Implement Safeguards:** Establish protocols to prevent the exchange of sensitive information that is not essential to the marketing effort.
- **Legal Consultation:** Given the complexities of antitrust laws, consult with a legal professional to ensure the collaboration complies with all legal requirements.

**Conclusion:**

While joint marketing campaigns between competitors are not inherently illegal, they must be structured carefully to avoid antitrust pitfalls. Legal guidance is essential to navigate these issues and to design a collaboration that achieves your business objectives without violating antitrust laws.
</response>
</example>""")

In [20]:
prompt = example_prompt + legal_query
print(prompt)

<prompt>You are a lawyer specializing in competition law, assisting business owners with their questions.</prompt>
<policy>As a legal professional, provide clear and accurate information about competition law while maintaining confidentiality and professionalism. Avoid giving specific legal advice without sufficient context, and encourage clients to seek personalized counsel when necessary.</policy>
<example>
<question>
I'm considering collaborating with a competitor on a joint marketing campaign. Are there any antitrust issues I should be aware of?
</question>
<response>
Collaborating with a competitor on a joint marketing campaign can raise antitrust concerns under U.S. antitrust laws, particularly the Sherman Antitrust Act of 1890 (15 U.S.C. §§ 1–7). Section 1 of the Sherman Act prohibits any contract, combination, or conspiracy that unreasonably restrains trade or commerce among the states.

**Key Considerations:**

1. **Per Se Illegal Agreements:** Certain collaborations are consi

In [21]:
resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 572 | Completion tokens = 2470 | Total tokens = 3042
Total cost = 0.010942957480000001 EUR
Completed in 00:00:42

The practice you describe could raise antitrust concerns, but its legal implications will depend on several factors. In general, larger companies offering suppliers incentives (or financial inducements) not to do business with a competitor may be engaging in conduct that could be viewed as anticompetitive—especially if the larger company holds a significant market share or dominant position.

Here are some key points to consider:

1. Exclusive Dealing and Foreclosure Concerns:  
   Such incentives might be considered a form of exclusive dealing or tying if they effectively restrict the ability of suppliers to transact with competitors. Under U.S. antitrust law (for example, under the Clayton Act), exclusive dealing arrangements are scrutinized when they foreclose market access for competitors in a way that harms competition or consumer choice.

2. Market Pow

## Coding

In [22]:
prompt = "Can you provide a structured template and guide for setting up a Python Flask project that connects to a PostgreSQL database?"

resp = o3mini(prompt, reasoning_effort='medium')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 29 | Completion tokens = 2022 | Total tokens = 2051
Total cost = 0.008498264448000002 EUR
Completed in 00:00:31

Below is an example of a structured template and guide to set up a Python Flask project that connects to a PostgreSQL database. This guide uses Flask, Flask‑SQLAlchemy for ORM support, and psycopg2‑binary for connecting to PostgreSQL.

──────────────────────────────
Step 1. Create Your Project Structure

A common project organization is:

Project Folder (my_flask_app/)
│
├── app/
│   ├── __init__.py      ← Initialize Flask and SQLAlchemy here
│   ├── models.py        ← Define your database models here
│   └── routes.py        ← Define your Flask routes (views) here
│
├── config.py            ← Centralized configuration (DB URI, etc.)
├── run.py               ← Entry point to start the app
├── requirements.txt     ← List of dependencies
└── .env                 ← Environment variables (optional – for secrets)
──────────────────────────────

───────────────────

In [23]:
prompt = """
Can you convert this Python code to JavaScript (Node.js) while maintaining its logic and functionality?

from collections import Counter
# Sample text input
text = "This is a sample text. This text is meant to test word frequency. Some words appear more than once, some words appear only once."

# Convert to lowercase and split into words
words = text.lower().split()

# Count word occurrences
word_counts = Counter(words)

# Print results
for word, count in word_counts.items():
    print(f"{word}: {count}")

"""

resp = o3mini(prompt, reasoning_effort='medium')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 117 | Completion tokens = 554 | Total tokens = 671
Total cost = 0.002442586736 EUR
Completed in 00:00:08

Below is one way to convert your Python code into JavaScript (Node.js):

--------------------------------------------------
"use strict";

const text = "This is a sample text. This text is meant to test word frequency. Some words appear more than once, some words appear only once.";

// Convert to lowercase and split into words (splitting on whitespace)
const words = text.toLowerCase().split(/\s+/);

// Count word occurrences using an object
const wordCounts = {};

for (const word of words) {
  // If the word already exists, increment its count; otherwise, set it to 1
  if (wordCounts[word]) {
    wordCounts[word]++;
  } else {
    wordCounts[word] = 1;
  }
}

// Print the results
for (const [word, count] of Object.entries(wordCounts)) {
  console.log(`${word}: ${count}`);
}

--------------------------------------------------

Explanation:

1. The text is converted 

In [24]:
prompt = """
Can you add proper error handling to this Python function making an API request?

import requests
response = requests.get("https://api.example.com/data")
data = response.json()
"""

resp = o3mini(prompt, reasoning_effort='low')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 42 | Completion tokens = 568 | Total tokens = 610
Total cost = 0.002422692112 EUR
Completed in 00:00:08

Below is an updated version of your code that includes proper error handling using try/except blocks and checks for HTTP errors:

---------------------------------------------------
import requests

def fetch_data():
    url = "https://api.example.com/data"
    try:
        response = requests.get(url, timeout=10)  # Adding a timeout is also good practice.
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # You might want to log this error.
        return None
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
        return None
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
        return None
  

In [25]:
prompt = """
Can you optimize this Python code for performance? It currently has inefficiencies in checking for prime numbers and runs slowly when searching for primes in a large range (e.g., 1 to 100,000).

def is_prime(n):
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True


def find_primes(start, end):
    primes = []
    for num in range(start, end + 1):  
        if is_prime(num):  
            primes.append(num)  
    return primes

# Example usage
start_range = 1
end_range = 10000
primes = find_primes(start_range, end_range)
print(f"Found {len(primes)} prime numbers.")
"""

resp = o3mini(prompt, reasoning_effort='medium')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 175 | Completion tokens = 1190 | Total tokens = 1365
Total cost = 0.005166806960000001 EUR
Completed in 00:00:16

Below are two optimized approaches. The first approach improves the is_prime() check by:

• Stopping the loop at the square root of n instead of n – 1.  
• Checking even numbers separately so that only odd candidates are tested.

The second approach uses the Sieve of Eratosthenes algorithm which is significantly faster for generating all primes in a large range.

──────────────────────────────
Approach 1: Optimized is_prime()

import math

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    # Eliminate even numbers
    if n % 2 == 0:
        return False
    # Only check odd divisors up to sqrt(n)
    limit = int(math.sqrt(n)) + 1
    for i in range(3, limit, 2):
        if n % i == 0:
            return False
    return True

def find_primes(start, end):
    primes = []
    for num in range(start, end + 1):
        if 

## Training

In [26]:
prompt = """
Create a 60-day learning plan to become proficient in Python
programming for data analysis, targeting a beginner with no prior
programming experience. The plan should include daily learning
goals, relevant resources (articles, videos, tutorials,
documentation, interactive coding exercises - describe the types
of resources that would be helpful, such as online coding
platforms, interactive tutorials, and documentation for Python
libraries, and where they can typically be found online, such as
Codecademy, Coursera, edX, and official Python documentation),
practical exercises (coding challenges, creating different types of
programs for data manipulation and analysis, working with
datasets - describe the kinds of projects that would be
appropriate, such as analyzing a public dataset, building a simple
data visualization tool, or creating a program to automate data
cleaning), and self-assessment methods (quizzes, mini-projects,
creating a portfolio of data analysis projects - describe the kinds
of projects that would be appropriate, such as analyzing a
dataset and presenting the findings in a report or creating an
interactive data visualization). Focus on key areas of Python for
data analysis, including 1) basic syntax and data structures, 2)
working with libraries like NumPy and Pandas, 3) data cleaning
and preprocessing techniques, 4) data visualization with
Matplotlib and Seaborn, and 5) basic statistical analysis. Include
links to reputable online learning platforms and communities,
such as Analytics Vidhya, DataCamp, and Udemy.
"""

resp = o3mini(prompt, reasoning_effort='low')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 324 | Completion tokens = 2126 | Total tokens = 2450
Total cost = 0.009242669384000002 EUR
Completed in 00:00:27

Below is a detailed 60‐day learning plan designed for a complete beginner to become proficient in Python for data analysis. This plan outlines daily learning goals, types of resources to use, practical exercises, and self-assessment methods to build confidence and competence in key areas such as basic syntax and data structures; libraries (NumPy and Pandas); data cleaning/preprocessing; data visualization (Matplotlib and Seaborn); and basic statistical analysis.

──────────────────────────────
Overview by Phase

• Days 1–15: Python fundamentals, environment setup, and basic syntax  
• Days 16–30: Data structures, control flow, functions, and introduction to libraries (focus on NumPy)  
• Days 31–45: Mastering Pandas, data cleaning/preprocessing, and introductory projects  
• Days 46–60: Data visualization with Matplotlib/Seaborn, statistical analysis, projec

## Solving

In [27]:
prompt = "Solve for x in the equation 3x^2 + 5x - 2 = 0"

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 27 | Completion tokens = 770 | Total tokens = 797
Total cost = 0.0032529396800000006 EUR
Completed in 00:00:12

To solve the equation 3x² + 5x - 2 = 0, we can use the quadratic formula:

  x = (-b ± √(b² - 4ac)) / (2a)

Here, a = 3, b = 5, and c = -2.

Step 1: Calculate the discriminant:

  b² - 4ac = 5² - 4(3)(-2)
      = 25 + 24
      = 49

Step 2: Substitute the values into the formula:

  x = (-5 ± √49) / (2×3)
    = (-5 ± 7) / 6

Step 3: Solve for the two cases:

  Case 1 (using +):
    x = (-5 + 7) / 6
      = 2 / 6
      = 1/3

  Case 2 (using -):
    x = (-5 - 7) / 6
      = -12 / 6
      = -2

Thus, the solutions to the equation are:
  x = 1/3 and x = -2

An alternative method is factoring. Notice that 3x² + 5x - 2 can be factored into (3x - 1)(x + 2) since:

  (3x - 1)(x + 2) = 3x² + 6x - x - 2 = 3x² + 5x - 2

Setting each factor equal to zero:

  3x - 1 = 0 ⟶ x = 1/3
  x + 2 = 0 ⟶ x = -2

Both methods confirm that the solutions are x = 1/3 and x = -2.


## Physic example

In [28]:
prompt = """ 
A ball is thrown vertically upward at the same instant that a second ball is dropped from rest directly above it.
The two balls are 12.0 meters apart when they start their motion.

Find the maximum speed at which the first ball can be throw such that it doesn't collide with the second ball before it returns to its starting height. Treat the balls as being very small (i.e. ignore their diameters).
"""

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 90 | Completion tokens = 3130 | Total tokens = 3220
Total cost = 0.013202306920000002 EUR
Completed in 00:00:39

Let the thrown ball’s initial speed be v₀. Choose a coordinate system with y = 0 at the lower (throwing) point, and y increasing upward. At t = 0:

• Thrown ball: y₁(0) = 0, with initial upward speed v₀.
• Falling ball: y₂(0) = 12.0 m (12.0 m directly above) and is dropped from rest.

Under gravity (take acceleration g = 9.80 m/s²), the positions as functions of time are:

 y₁(t) = v₀t – ½ g t²  
 y₂(t) = 12.0 m – ½ g t²

Notice that both positions have the same –½g t² term. The collision occurs when y₁(t) = y₂(t). Setting them equal gives

 v₀t – ½ g t² = 12.0 m – ½ g t²  
 ⇒ v₀t = 12.0 m  
 ⇒ t_collision = 12.0 / v₀   (1)

The thrown ball returns to its starting height (y = 0) when

 y₁(t) = v₀t – ½ g t² = 0  
 ⇒ t = 0  or  t = 2v₀/g

Ignoring the trivial t = 0, the return time is

 t_return = 2v₀/g   (2)

To avoid a collision before the thrown ball returns

## Math example

In [29]:
prompt = "Prove the Pythagorean theorem using a geometric approach."

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 19 | Completion tokens = 1243 | Total tokens = 1262
Total cost = 0.0052254328120000006 EUR
Completed in 00:00:18

One of the most elegant geometric proofs of the Pythagorean theorem uses a rearrangement argument. Follow these steps to see how the areas work out:

1. Imagine a right triangle with legs of length a and b and hypotenuse of length c. We want to show that a² + b² = c².

2. Construct a large square whose side length is (a + b). Its total area is (a + b)².

3. Inside this large square, arrange four identical copies of the right triangle in such a way that their hypotenuses form a smaller, rotated square in the center. (You can picture each triangle placed in the corners of the large square, with the right angle of each triangle touching a corner. The triangles’ hypotenuses then line the inner square.)

   • In this configuration, the four triangles do not overlap, and the region left uncovered in the center is a smaller square. By examining the arrangement (or 

## Programming example

In [30]:
prompt = "Write a Python program that shows a ball bouncing inside a spinning hexagon. The ball should be affected by gravity and friction, and it must bounce off the rotating walls realistically"

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 40 | Completion tokens = 7003 | Total tokens = 7043
Total cost = 0.029369631652000006 EUR
Completed in 00:01:28

Below is one complete Python program that uses pygame to simulate a ball bouncing inside a spinning hexagon. In this example the ball is a circle that “falls” under gravity, loses a bit of energy (simulating friction) and bounces off the rotating hexagon’s walls with a collision response computed relative to the wall’s own velocity. (You can adjust parameters such as gravity, friction and restitution for different “feels.”)

To try it out, make sure you have pygame installed (for example by running “pip install pygame”), then save the code to a file (say “bouncing_ball_hexagon.py”) and run it.

------------------------------------------------------------
#!/usr/bin/env python3
"""
A simulation using pygame where a ball bounces inside a spinning hexagon.
The ball is subject to gravity and some air friction.
When it hits the hexagon walls (which are moving beca

## Optimization

In [31]:
prompt = """
x and y are integers.
You need to maximize this function: 3x + 2y

with these five contraints:
2x + y <= 18
2x + 3y <= 42
3x + y <= 24
x >= 0
y >= 0

Print the results of this optimal problem.
"""

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 78 | Completion tokens = 4116 | Total tokens = 4194
Total cost = 0.017318996544000004 EUR
Completed in 00:02:21

We want to maximize

  Objective: 3x + 2y

subject to the constraints

  (1) 2x + y ≤ 18  
  (2) 2x + 3y ≤ 42  
  (3) 3x + y ≤ 24  
  (4) x ≥ 0  
  (5) y ≥ 0

with x and y being integers.

Step 1. Find the intersection points (vertices) of the boundaries of the constraints (ignoring that x and y must be integers for now). These intersections are often candidates for an optimal solution.

• Intersection of (1) and (3):

  2x + y = 18  
  3x + y = 24  
Subtract the first from the second:  
  (3x + y) − (2x + y) = 24 − 18  
  x = 6  
Now substitute x = 6 into (1):  
  2(6) + y = 18  
  12 + y = 18 ⇒ y = 6  
Thus, one vertex is (6, 6).

• Intersection of (1) and (2):

  2x + y = 18  
  2x + 3y = 42  
Subtract the first from the second:  
  (2x + 3y) − (2x + y) = 42 − 18  
  2y = 24 ⇒ y = 12  
Now substitute y = 12 into (1):  
  2x + 12 = 18 ⇒ 2x = 6 ⇒ x = 3  
Thi

In [60]:
prompt = """
300 kids need to travel to the Bronx zoo.
The school may rent 40 seats and 30 seats buses for $500 and $400.

How many buses of each size do you use to minimize cost?
Print the number of 40 seats buses and the number of 30 seats buses and the total cost.
"""

resp = o3mini(prompt)
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 70 | Completion tokens = 1555 | Total tokens = 1625
Total cost = 0.006585449620000001 EUR
Completed in 00:00:19

Let x = number of 40‐seat buses and y = number of 30‐seat buses. The school needs at least 300 seats, so the constraint is

  40x + 30y ≥ 300.

The cost function is

  Cost = 500x + 400y.

Since a 40‐seat bus costs $500 and a 30‐seat bus costs $400, note that the cost per seat is higher for the 30‐seat bus (500/40 = 12.5 per seat versus 400/30 ≈ 13.33 per seat). This suggests that using as many 40-seat buses as possible is more economical, but we must meet the exact seating requirement.

One way to proceed is to try integer combinations that give at least 300 seats.

1. Using only 40‐seat buses:
  We need x such that 40x ≥ 300, so x = 8 (since 8 × 40 = 320).
  Cost = 8 × 500 = $4000.

2. Try a mix:
  If we use 6 40-seat buses, that gives 6 × 40 = 240 seats.
  Then we need 300 – 240 = 60 more seats. Each 30-seat bus adds 30 seats, so we need y = 2.
  Total sea

In [61]:
prompt = """
A train X left at 11:58 and travels at a speed of 210 km/h.
A train Y left at 13:10 but travels at 380 km/h.

At what time does train Y catch up with train X?
"""

resp = o3mini(prompt)
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 56 | Completion tokens = 1196 | Total tokens = 1252
Total cost = 0.005067341264000001 EUR
Completed in 00:00:25

Step 1: Determine the head start of Train X.
• Train X departs at 11:58.
• Train Y departs at 13:10.
• The time difference is 1 hour and 12 minutes, which is 1.2 hours.

Step 2: Calculate the lead distance.
• Speed of Train X = 210 km/h.
• Distance = speed × time = 210 km/h × 1.2 h = 252 km.

Step 3: Set up the catch-up equation.
When Train Y starts at 13:10, it must cover the 252 km head start plus the distance that Train X travels afterwards.
Let t be the time (in hours) after 13:10 when Train Y catches up with Train X.
• Distance traveled by Train Y = 380 km/h × t.
• Distance traveled by Train X after 13:10 = 210 km/h × t.
Since Train X already has a 252 km lead, at catch-up the distances are equal:
  380t = 252 + 210t

Step 4: Solve for t.
  380t - 210t = 252
  170t = 252
  t = 252/170 ≈ 1.48235 hours

Step 5: Convert t into hours, minutes, and seconds.
•

## Chemical

In [34]:
prompt = """
Sodium hydroxide (NaOH) reacts with phosphoric acid (H₃PO₄) to form sodium phosphate (Na₃PO₄) and water (H₂O) by the reaction:
3 NaOH(aq) + H₃PO₄(aq) → Na₃PO₄(aq) + 3 H₂O(l) If 35.60 grams of NaOH are reacted with 30.80 grams of H₃PO₄.
How many grams of Na₃PO₄ are formed?
"""

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 123 | Completion tokens = 1596 | Total tokens = 1719
Total cost = 0.006812643864 EUR
Completed in 00:00:19

Step 1: Write the balanced reaction.
  3 NaOH(aq) + H₃PO₄(aq) → Na₃PO₄(aq) + 3 H₂O(l)

Step 2: Calculate moles of reactants.

For NaOH:
  Molar mass of NaOH ≈ 40.00 g/mol
  Moles of NaOH = 35.60 g ÷ 40.00 g/mol = 0.89 mol

For H₃PO₄:
  Molar mass of H₃PO₄ ≈ 98.00 g/mol
  Moles of H₃PO₄ = 30.80 g ÷ 98.00 g/mol ≈ 0.3143 mol

Step 3: Determine the limiting reactant.
  The balanced equation shows that 3 moles of NaOH react with 1 mole of H₃PO₄.
  To completely react with 0.3143 mol H₃PO₄, you would need:
    0.3143 mol H₃PO₄ × 3 mol NaOH/mol H₃PO₄ = 0.9429 mol NaOH
  We only have 0.89 mol NaOH, so NaOH is the limiting reactant.

Step 4: Calculate moles of Na₃PO₄ produced.
  According to the reaction, 3 moles of NaOH produce 1 mole of Na₃PO₄.
  Moles of Na₃PO₄ = 0.89 mol NaOH ÷ 3 = 0.2967 mol

Step 5: Convert moles of Na₃PO₄ to grams.
  Molar mass of Na₃PO₄:
   Na: 22.

## Transportation

In [35]:
prompt = """
Design an integrated strategy to optimize urban transportation in a rapidly growing megacity. 
Your plan should address the following aspects: security, unified governance, multi-modal transit, sustainability.
"""

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 42 | Completion tokens = 2799 | Total tokens = 2841
Total cost = 0.011765861316000001 EUR
Completed in 00:00:36

Below is a comprehensive strategy that integrates security, unified governance, multi‐modal transit, and sustainability to optimize urban transportation in a rapidly growing megacity.

─────────────────────────────  
1. Establish a Unified Governance Framework

• Create a Central Transportation Authority (CTA):  
 – Form a single governing body that brings together the city’s transportation planners, law enforcement, environmental agencies, and IT/security teams.  
 – Empower this authority to coordinate planning, budgeting, operations, and crisis management across all transit modes and infrastructure components.

• Develop an Integrated Digital Platform:  
 – Build a citywide control center that aggregates real-time data from all transportation modes, monitoring systems, and environmental sensors.  
 – Use this platform to enable data-driven decision-making,

## Finance analysis

In [36]:
prompt = """
I would like a thorough analysis of IBM stock, including:

- Summary: Company overview, key metrics, performance data and investment recommendations.
- Financial Data: Revenue trends, profit margins, balance sheet and cash flow analysis.
- Market Sentiment: Analyst ratings, sentiment indicators and news impact.
- Technical Analysis: Price trends, technical indicators and support/resistance levels.
- Compare Assets: Market share and financial metrics vs. key competitors.
- Value Investor: Intrinsic value, growth potential and risk factors.
- Investment Thesis: SWOT analysis and recommendations for different investor types.
"""

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 121 | Completion tokens = 5172 | Total tokens = 5293
Total cost = 0.021786423048000002 EUR
Completed in 00:01:14

Below is a comprehensive analysis of IBM stock broken into several key sections. Please note that while this analysis is based on recent trends and publicly available data as of the latest period, investors should verify current figures and consider their own research before making any decisions.

──────────────────────────────
1. Summary

• Company Overview:  
 – IBM (International Business Machines Corporation) is a legacy technology giant with global reach.  
 – The company is known for its hardware, middleware, software, IT consulting, and services. In recent years, IBM has reoriented its strategy toward hybrid cloud, artificial intelligence (AI), and quantum computing while gradually winding down some of its legacy hardware and software segments.  
 – A key element of its transformation is the integration of acquisitions—most notably Red Hat—to build a 

## SWOT analysis

In [37]:
prompt = "Generate a SWOT analysis for IBM"

resp = o3mini(prompt, reasoning_effort='medium')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 14 | Completion tokens = 1027 | Total tokens = 1041
Total cost = 0.004315614868000001 EUR
Completed in 00:00:14

Below is a SWOT analysis for IBM:

Strengths
• Strong Brand and Legacy – IBM’s long history and well-established global reputation create significant trust among enterprise customers.
• Diversified Technology Portfolio – The company offers a broad range of products and services, from legacy mainframes to modern cloud computing, artificial intelligence (e.g., Watson), quantum computing, and cybersecurity.
• Robust Research and Development – IBM invests heavily in R&D, maintaining a leadership position in emerging technologies.
• Established Client Relationships – Long-term relationships with large enterprises and government organizations provide stable revenue sources.
• Global Operations and Strategic Alliances – IBM’s worldwide presence and partnerships with other tech and consulting firms enhance its market reach.

Weaknesses
• Legacy System Challenges – A 

## Aerospace

In [38]:
prompt = """
You are an advanced AI assistant with expertise in aerospace engineering, aerodynamics, propulsion systems, avionics, and space exploration. You have strong reasoning capabilities and can analyze complex aerospace problems, optimize designs, and evaluate trade-offs.

Given the following aerospace scenario or problem statement, provide a detailed response that includes:

Technical Analysis – Apply aerospace engineering principles to break down the problem.
Logical Reasoning – Use step-by-step deductive or inductive reasoning to evaluate solutions.
Optimization & Trade-offs – Compare different approaches, considering efficiency, cost, safety, and feasibility.
Innovative Thinking – Suggest novel or cutting-edge solutions, incorporating recent advancements in aerospace technology.
Ensure your response is well-structured, technically sound, and provides explanations that are understandable to both engineers and decision-makers. If relevant, include references to established aerospace standards (e.g., NASA, FAA, ESA) and scientific principles.
"""

resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 183 | Completion tokens = 3780 | Total tokens = 3963
Total cost = 0.016021802520000003 EUR
Completed in 00:00:52

Below is an example of how one might address a challenging aerospace design problem—for instance, developing a next‐generation reusable orbital launch vehicle—and perform a detailed technical analysis. (If you have a more specific scenario in mind, please let me know, and I can tailor the discussion further.) 

──────────────────────────────
1. Problem Statement Example

Imagine the following design challenge: 
• Develop a reusable first-stage booster for an orbital launch system that can deliver a 10–15‐ton payload to low Earth orbit while meeting strict safety, cost, and turnaround time targets.  
• The design must optimize aerodynamic performance, propulsion efficiency, thermal protection, and avionics to enable rapid reuse with minimal refurbishment between flights.

──────────────────────────────
2. Technical Analysis

A. Mission and Vehicle Requirement

## Predictive maintenance

In [39]:
prompt = """
Proactive Maintenance Planning for Contoso: Advanced Transformer Analysis

Background
Contoso, a leading energy provider, is undertaking a comprehensive analysis of its electrical distribution network to optimize maintenance planning for its critical transformers. The goal is to proactively identify transformers at risk of failure due to thermal stress, insulation degradation, overloading, and other factors, especially under projected future operating conditions. This analysis requires detailed examination of operational parameters, environmental conditions, diagnostic test results, and adherence to safety regulations.
 
Datasets
Overview
 
You are provided with detailed datasets for seven transformers (T1001 to T1007). Your task is to analyze these datasets to determine which transformer(s) require immediate maintenance or action within the next six months. The data includes:

Asset Inventory and Specifications
Load and Operational Data
Environmental Conditions
Transformer Thermal Performance Data
Insulation Deterioration Rates
Transformer Loading Guide
Technician Reports
Dissolved Gas Analysis (DGA) Results
Safety Regulations and Maintenance Constraints
Projected Future Operations
 
Dataset 1: Asset Inventory and Specifications
 
Asset Inventory Table
Asset ID	Location	Install Date	Expected Lifespan (years)	Last Maintenance Date	Manufacturer	Model	Voltage Ratio (kV/kV)	Power Rating (MVA)	Impedance (%)	Cooling Type	Criticality Score (1-10)	Expected Retirement Date
T1001	123 Main St	2005-06-15	25	2018-05-20	ElectroTrans Co.	ETX-2000	220/110	100	8	ONAN	9	2030-06-15
T1002	456 Oak Ave	2010-08-22	25	2017-11-10	PowerMax Solutions	PMX-1500	220/110	90	9	ONAF	7	2035-08-22
T1003	789 Industrial Rd	2000-03-30	25	2015-09-05	VoltGear Ltd.	VG-3000	220/110	120	10	ONAN	10	2025-03-30
T1004	321 Maple Dr	1995-01-12	25	2014-04-18	ElectroTrans Co.	ETX-1800	110/33	80	7	ONAN	6	2020-01-12
T1005	654 Pine St	2007-12-05	25	2016-07-25	PowerMax Solutions	PMX-1600	220/110	109	6	ONAF	8	2032-12-05
T1006	987 Elm St	1990-11-20	25	2013-03-15	VoltGear Ltd.	VG-2500	220/110	90	8	ONAN	5	2015-11-20
T1007	222 Cedar Rd	2008-09-10	25	2019-02-22	ElectroTrans Co.	ETX-2200	220/110	110	7	ONAF	8	2033-09-10
 												
Additional Notes:												
 												
Impedance (%) affects voltage regulation and fault levels.
Cooling Types:
ONAN: Oil Natural Air Natural
ONAF: Oil Natural Air Forced
 
Dataset 2: Load and Operational Data
Monthly Average Load Data (Last 12 Months)
 
Transformer T1001
 
Month	Average Load (MVA)
Month -11	80
Month -10	82
Month -9	84
Month -8	85
Month -7	87
Month -6	88
Month -5	90
Month -4	92
Month -3	90
Month -2	88
Month -1	90
Current Month	92
 	
---	
 	
#### Transformer T1002	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	66
Month -10	67
Month -9	68
Month -8	69
Month -7	68
Month -6	70
Month -5	71
Month -4	70
Month -3	69
Month -2	70
Month -1	69
Current Month	70
 	
---	
 	
#### Transformer T1003	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	110
Month -10	112
Month -9	114
Month -8	116
Month -7	116
Month -6	116
Month -5	116
Month -4	116
Month -3	118
Month -2	115
Month -1	116
Current Month	117
 	
---	
 	
#### Transformer T1004	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	60
Month -10	61
Month -9	62
Month -8	63
Month -7	62
Month -6	64
Month -5	65
Month -4	66
Month -3	65
Month -2	64
Month -1	65
Current Month	65
 	
---	
 	
#### Transformer T1005	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	90
Month -10	91
Month -9	92
Month -8	93
Month -7	94
Month -6	95
Month -5	96
Month -4	95
Month -3	96
Month -2	95
Month -1	95
Current Month	95
 	
---	
 	
#### Transformer T1006	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	88
Month -10	88
Month -9	88
Month -8	87
Month -7	87
Month -6	86
Month -5	86
Month -4	85
Month -3	85
Month -2	85
Month -1	85
Current Month	85
 	
---	
 	
#### Transformer T1007	
 	
Month	Average Load (MVA)
------------	-------------------
Month -11	95
Month -10	96
Month -9	97
Month -8	98
Month -7	99
Month -6	100
Month -5	101
Month -4	100
Month -3	99
Month -2	100
Month -1	100
Current Month	100
 	
 
Projected Load Changes:
T1001: Expected to remain relatively stable with minor fluctuations due to seasonal demand variations.
T1002: Anticipated to maintain current load levels with minor fluctuations. However, a significant load increase of 15 MVA is expected in Month +4 due to the connection of a new residential development.
T1003: Expected to increase by 1% per month due to new industrial expansion starting immediately.
T1004: Expected to remain relatively stable with minor fluctuations, maintaining its current load levels.
T1006: Expected to experience a slight decrease in load by -1% per month due to energy efficiency programs implemented in the service area.
T1007: Anticipated to maintain relatively stable load levels with minor fluctuations typical of regular operational demands.
 
Notes:

The transformers T1001, T1002 (until Month +4), T1004, T1005, and T1007 are expected to have stable load profiles, with only minor fluctuations that are characteristic of normal seasonal variations and operational demands.
T1002 will experience a significant load increase in Month +4 due to the upcoming new residential development, which is projected to add 15 MVA to its current load.
The load increase for T1003 due to industrial expansion and the decrease for T1006 due to energy efficiency measures are significant and should be considered in operational planning.
All projections consider current known factors; unexpected changes in consumer behavior or unforeseen events are not accounted for in these projections.


Dataset 3: Environmental Conditions
Ambient Temperature Data (Historical and Forecasted)

Ambient Temperatures (ÃÂ¡C):

Month	Ambient Temp (ÃÂ¡C)
Month -11	25
...	...
Current Month	40
Month +1	41
Month +2	42
Month +3	43
Month +4	42
Month +5	40
Month +6	38
 	
Notes:	
 	
Ambient temperatures peak in Month +3.
High temperatures can influence transformer cooling efficiency and insulation aging.
 
Dataset 4: Transformer Thermal Performance Data
Cooling Efficiency Coefficients

Model	Cooling Type	CEC	Rated ___TO-R (ÃÂ¡C)	Rated ___HS-R (ÃÂ¡C)
VG-3000	ONAN	1.0	55	30
PMX-1500	ONAF	0.85	50	25
ETX-2000	ONAN	0.95	55	30
VG-2500	ONAN	0.9	55	30
PMX-1600	ONAF	0.88	50	25
ETX-1800	ONAN	0.92	55	30
ETX-2200	ONAF	0.86	50	25
 				
Exponents for Temperature Calculations				
 				
ONAN Cooling:
( m = 0.8 )
( n = 1.6 )
ONAF Cooling:
( m = 0.9 )
( n = 1.8 )
 
Dataset 5: Insulation Deterioration Rates
Insulation Life Expectancy vs. Hot-Spot Temperature

Hot-Spot Temperature (ÃÂ¡C)	Relative Aging Rate
110	1
120	2
130	4
140	8
150	16
 	
---	
 	
### Dataset 6: Transformer Loading Guide	
	
IEEE Standard Formulas	
 	
Top-Oil Temperature Rise (( \Delta \theta_{\text{TO}} )):

[
\Delta \theta_{\text{TO}} = \Delta \theta_{\text{TO-R}} \times \left( \frac{L}{L_{\text{R}}} \right)^{m}
]

Hot-Spot Temperature Rise (( \Delta \theta_{\text{HS}} )):

[
\Delta \theta_{\text{HS}} = \Delta \theta_{\text{HS-R}} \times \left( \frac{L}{L_{\text{R}}} \right)^{n}
]

Total Hot-Spot Temperature (( \theta_{\text{HS}} )):

[
\theta_{\text{HS}} = \theta_{\text{Amb}} + \Delta \theta_{\text{TO}} + \Delta \theta_{\text{HS}}
]

Where:

( L ): Load in MVA
( L_{\text{R}} ): Rated Load in MVA
( m ), ( n ): Exponents based on cooling type
 

Dataset 7: Technician Reports

T1001 Report:

No significant issues observed. Regular operational parameters.

T1002 Report:
  Cooling fans operating efficiently. Oil color slightly darker than normal; recommend oil test.  

T1003 Report:
  Transformer oil shows higher than normal moisture content.  
  DGA indicates increased levels of ethylene and methane.  
  Cooling system operating at maximum capacity.  

T1004 Report:
  Signs of aging observed. Minor oil leaks detected.  

T1005 Report:
  External overheating signs on components.  
  Load readings higher during peak hours.  

T1006 Report:
  Transformer operating past expected lifespan.  
  Cooling fins corroded; reduced cooling efficiency expected.  

T1007 Report:
  All parameters within normal limits. No action required.  
 

Dataset 8: Dissolved Gas Analysis (DGA) Results
 
Gas Levels in Parts Per Million (ppm)

Gas	T1001	T1002	T1003	T1004	T1005	T1006	T1007
Hydrogen (H2)	20	40	150	30	50	80	25
Methane (CH4)	10	25	120	15	30	60	12
Ethylene (C2H4)	5	15	90	8	20	40	5
Ethane (C2H6)	4	10	60	6	15	35	4
Acetylene (C2H2)	0	0	5	0	0	0	0
Carbon Monoxide (CO)	50	80	300	70	90	200	60
Carbon Dioxide (CO2)	200	400	1200	250	450	800	220
 							
Interpretation Standards:							
 							
Action Levels:
H2 > 100 ppm
CH4 > 75 ppm
C2H4 > 50 ppm
CO > 350 ppm
Presence of Acetylene (C2H2) indicates high-temperature arcing.
High levels of CO and CO2 suggest cellulose insulation degradation.
 
Dataset 9: Safety Regulations and Maintenance Constraints
 
Regulatory Guidelines:

Maintenance must not be performed when ambient temperatures exceed 40ÃÂ¡C due to safety concerns.
Transformers operating beyond 80% of their expected lifespan require thorough inspections.
DGA action levels necessitate immediate investigation and potential maintenance.
Ambient Temperature Maintenance Window: Best scheduled during months with ambient temperatures below 40ÃÂ¡C.

Dataset 11: SME reports
Subject: Analysis of Time-Variant Load Impact on Transformer T1005 Due to New Industrial Park Opening
 
Establishments Overview:
 

MetalWorks Manufacturing Co.
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
GreenTech Recycling Facility
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
FreshFoods Processing Plant
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
LogiTrans Distribution Center
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Quantum Data Centers Ltd.
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging
 
Time-Variant Load Profiles:
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):
Time Window	MetalWorks (MW)	GreenTech (GT)	FreshFoods (FF)	LogiTrans (LT)	Quantum Data (QD)
1 (00-04)	1.0	0.5	0.0	0.0	2.5
2 (04-08)	2.0	1.0	0.0	0.0	2.5
3 (08-12)	4.0	3.0	2.5	0.0	3.0
4 (12-16)	4.0	3.0	2.5	3.5	3.0
5 (16-20)	2.0	1.5	1.0	3.5	2.0
6 (20-24)	1.0	0.5	0.0	1.0	2.5
 					

1. Upcoming Commercial Complex Affecting Transformer T1001
 
Establishments Overview:

Sunrise Retail Plaza
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
EcoWise Offices
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
Harvest Foods Market
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
Swift Logistics Hub
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Nimbus Data Centers
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging

Time-Variant Load Profiles:
 
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):
Time Window	Sunrise Retail	EcoWise Offices	Harvest Foods	Swift Logistics	Nimbus Data
1 (00-04)	0.5	0.3	0.0	0.0	0.8
2 (04-08)	0.8	0.5	0.0	0.0	0.8
3 (08-12)	1.0	0.6	0.5	0.0	0.9
4 (12-16)	1.0	0.6	0.5	0.5	0.9
5 (16-20)	0.6	0.5	0.2	0.5	0.6
6 (20-24)	0.5	0.3	0.0	0.2	0.8
 					
---

2. Residential Development Impacting Transformer T1002
 
Establishments Overview:
 
Grandview Shopping Center
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
Harmony Business Park
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
FreshMart Grocery
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
Express Logistics Depot
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Apex Data Center
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging

Time-Variant Load Profiles:
 
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):
Time Window	Grandview Mall	Harmony Business	FreshMart Grocery	Express Logistics	Apex Data Center
1 (00-04)	0.5	0.3	0.0	0.0	0.8
2 (04-08)	0.8	0.5	0.0	0.0	0.8
3 (08-12)	1.0	0.6	0.5	0.0	0.9
4 (12-16)	1.0	0.6	0.5	0.5	0.9
5 (16-20)	0.6	0.5	0.2	0.5	0.6
6 (20-24)	0.5	0.3	0.0	0.2	0.8
 					
---		

3. Small Industrial Unit Connecting to Transformer T1004
 
Overview:

A small-scale manufacturing unit is set to commence operations near Transformer T1004:

Precision Tools Ltd.: Specializing in custom tool manufacturing.
Expected Load Increase: Approximately 1.8 MVA.

4. New School Facility Supplied by Transformer T1005
 
Overview:
A new educational institution is opening in the area served by Transformer T1005:
Sunrise Elementary School: Modern facilities with standard energy consumption.
Expected Load Increase: Approximately 1.2 MVA.

5. Community Center Addition Affecting Transformer T1007
 
Overview:
Construction of a community center is nearing completion in the vicinity of Transformer T1007:
Riverdale Community Hall: Hosting events and activities for local residents.
Expected Load Increase: Approximately 1 MVA.

6. Small Business Cluster Connecting to Transformer T1003
 
Overview:
A cluster of small businesses is opening near Transformer T1003:

Shops Include:
Cafe Delight: Boutique coffee shop.
Health Rx Pharmacy: Local pharmacy outlet.
Bloom Floral Shop: Florist with refrigeration needs.
TechFix IT Services: Electronics repair shop.
FitLife Gym Studio: Fitness center with standard equipment.
Expected Load Increase: Approximately 2 MVA combined.
 

Question
Task
 
As an analyst at Contoso, your tasks are:

Identify the transformer(s) that should be prioritized for maintenance or immediate action within the next six months, based on the data provided.
For each transformer you identify, provide a detailed rationale explaining why it should be prioritized, including all necessary calculations, engineering considerations, and interpretations of the DGA results.
Recommend a specific maintenance or replacement plan for these transformer(s), considering operational constraints, safety regulations, projected load changes, and potential risks.

Note: Your analysis should aim to:
Prevent potential transformer failures due to thermal faults, insulation degradation, or overloading.
Utilize electrical engineering principles and standards for transformer diagnostics.
Ensure compliance with safety regulations and operational constraints.
Optimize maintenance scheduling by considering ambient temperature constraints and load projections.
Focus on one or two transformers that critically need attention based on the data.
 
Instructions:
Thoroughly analyze the provided datasets, performing all necessary calculations to assess the condition and future performance of the transformers.
Apply electrical engineering principles, including calculations of hot-spot temperatures, interpretation of DGA results, and understanding of thermal impacts on transformer insulation.
Include all intermediate calculations and reasoning steps in your analysis.
Reference specific data points from the datasets in your rationale.
Provide a brief executive summary at the beginning of your solution, highlighting your key findings and recommendations.
Present your findings in a clear, logical, and concise manner, suitable for presentation to the management team.
Identify only the transformer(s) that genuinely require immediate action, ensuring that your conclusions are based on solid evidence from the data.
  
"""

In [40]:
resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 4899 | Completion tokens = 8222 | Total tokens = 13121
Total cost = 0.039562035248000005 EUR
Completed in 00:01:42

Below is an executive summary followed by the detailed engineering analysis and recommendations.

──────────────────────────────
Executive Summary

After a thorough review of Contoso’s transformer datasets, two units stand out as requiring immediate attention within the next six months. Transformer T1003 (a 120‐MVA VG‑3000 unit) exhibits significant signs of thermal stress and insulation degradation – its dissolved gas analysis (DGA) readings far exceed action levels while projected load increases will push its operating temperature well into a regime that accelerates insulation aging. In addition, Transformer T1006 (a 90‐MVA VG‑2500 unit) has long exceeded its designed service life (retired in 2015) and shows clear physical deterioration (corroded cooling fins) that undermines its cooling efficiency. Our recommendation is to prioritize immediate maintenan

## Energy consumption

In [41]:
prommpt = """
scenario
A utility company is working with a residential neighborhood consisting of 100 households to optimize energy consumption. The following data is provided:

- **Time-of-use (TOU) Pricing:**

    | Time Period         | Price per kWh ($) | Renewable Energy Availability (%) |
    |---------------------|-------------------|-----------------------------------|
    | 12:00 AM - 6:00 AM  | 0.08              | 30%                               |
    | 6:00 AM - 10:00 AM  | 0.12              | 50%                               |
    | 10:00 AM - 4:00 PM  | 0.10              | 80%                               |
    | 4:00 PM - 9:00 PM   | 0.20              | 40%                               |
    | 9:00 PM - 12:00 AM  | 0.15              | 60%                               |

- **Household Appliance Data**:

    Each household has the following appliances with their average daily usage times and flexibility:

    | Appliance          | Average Usage (hours/day) | Usage Time      | Flexibility  |
    |--------------------|---------------------------|-----------------|--------------|
    | Washing Machine    | 1                         | Anytime         | High         |
    | Dishwasher         | 1                         | Evening         | Medium       |
    | HVAC System        | 8                         | Daytime/Night   | Low          |
    | Electric Vehicle   | 2                         | Night           | Medium       |
    | Water Heater       | 3                         | Morning/Evening | Medium       |

- **Constraints**:

    - **Resident Comfort**: HVAC adjustments must maintain indoor temperatures between 68ÃÂ¬Ã¢ÂÂF and 75ÃÂ¬Ã¢ÂÂF.
    - **Appliance Flexibility**:
        - High: Can be scheduled at any time within a 24-hour window.
        - Medium: Can be shifted within a 6-hour window.
        - Low: Cannot be rescheduled.

- **Objective**:

    - Minimize electricity costs for each household.
    - Maximize the use of renewable energy.
    - Ensure no impact on resident comfort and appliance performance.

</scenario>

<additional_information>
The utility company offers the following incentives:

- **Renewable Energy Credit**: Households that consume more than 50% of their electricity during high renewable availability periods (when availability is above 60%) receive a $10 monthly credit.
- **Off-Peak Discount**: An additional 5% discount on electricity prices during off-peak hours (12:00 AM - 6:00 AM and 10:00 AM - 4:00 PM).

</additional_information>
"""

In [42]:
resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 4899 | Completion tokens = 7385 | Total tokens = 12284
Total cost = 0.036056776340000006 EUR
Completed in 00:01:45

Below is the executive summary followed by the complete analysis with all calculations and supporting engineering reasoning.

──────────────────────────────
Executive Summary

After evaluating all operational, thermal, and diagnostic data for Contoso’s transformer fleet, our analysis shows that Transformer T1003 and Transformer T1006 must be given top priority for maintenance action within the next six months. 

• Transformer T1003 (VoltGear Ltd. VG‐3000) is operating near its rated load—with an upward trend from ongoing industrial expansion—and exhibits multiple diagnostic “red flags”: its dissolved gas analysis (DGA) shows hydrogen, methane, and ethylene levels that exceed industry action levels. Its computed hot‐spot temperatures (even under current ambient conditions) are “alarm‐level” and will worsen as ambient temperatures peak. 

• Transformer T1006

## Aerospace maintenance

In [43]:
prompt = """
You are an expert aerospace engineer with deep knowledge of aircraft design, propulsion systems, avionics, aerodynamics, and space technologies.
Given the following aerospace problem, provide a detailed analysis, reasoning through the physics, engineering constraints, and industry standards to arrive at a well-supported conclusion.

Problem Statement:
Aerospace engineers are designing a next-generation supersonic passenger aircraft.
The primary challenges include minimizing sonic boom impact, optimizing fuel efficiency, and ensuring regulatory compliance for commercial viability. 

Given these constraints:

Sonic Boom Mitigation:
- What are the most effective aerodynamic shaping techniques to reduce sonic boom intensity?
- How do computational fluid dynamics (CFD) simulations guide the optimization of fuselage and wing design?

Fuel Efficiency & Propulsion:
- Compare the feasibility of turbofan vs. turbojet vs. hybrid propulsion for supersonic flight.
- How can alternative fuels (e.g., sustainable aviation fuel, hydrogen) be integrated to balance performance and environmental impact?

Regulatory & Safety Considerations:
- What are the key FAA and ICAO regulations affecting supersonic travel?
- How should the aircraft be designed to meet noise and emissions standards while maintaining economic viability?

Evaluation Criteria:
- Technical Depth: Does the reasoning demonstrate expert-level aerospace engineering knowledge?
- Logical Coherence: Are the arguments structured logically with well-supported evidence?
- Innovation & Feasibility: Are the proposed solutions innovative yet practical within current technological and economic constraints?
- Regulatory Awareness: Does the response consider relevant aerospace regulations and compliance requirements?
"""

In [44]:
resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 309 | Completion tokens = 3474 | Total tokens = 3783
Total cost = 0.014872232016 EUR
Completed in 00:00:59

Below is a detailed expert analysis that walks through the technical, engineering, and regulatory challenges for designing a next-generation supersonic passenger aircraft:

─────────────────────────────  
1. Sonic Boom Mitigation  
─────────────────────────────

A. Aerodynamic Shaping Techniques

• Area Rule and Smooth Cross‐Sectional Transitions  
  • The “area rule” is one of the most effective strategies: by ensuring the cross‐sectional area changes as gradually as possible along the aircraft’s length, shock wave formation is minimized.  
  • This typically leads to a “coke bottle” or “wasp waist” design that smooths the pressure distribution, reducing the peak overpressures responsible for the sonic boom.

• Refined Fuselage and Nose Cone Geometry  
  • An elongated, slender nose with a carefully contoured forebody reduces the abruptness of the pressure rise a

## Maintenance

In [45]:
prompt = """
<question>
 
As an analyst at Contoso, your tasks are:

Identify the transformer(s) that should be prioritized for maintenance or immediate action within the next six months, based on the data provided.
For each transformer you identify, provide a detailed rationale explaining why it should be prioritized, including all necessary calculations, engineering considerations, and interpretations of the DGA results.
Recommend a specific maintenance or replacement plan for these transformer(s), considering operational constraints, safety regulations, projected load changes, and potential risks.

Note: Your analysis should aim to:
Prevent potential transformer failures due to thermal faults, insulation degradation, or overloading.
Utilize electrical engineering principles and standards for transformer diagnostics.
Ensure compliance with safety regulations and operational constraints.
Optimize maintenance scheduling by considering ambient temperature constraints and load projections.
Focus on one or two transformers that critically need attention based on the data.
 
 </question>

<instructions>

Thoroughly analyze the provided datasets, performing all necessary calculations to assess the condition and future performance of the transformers.
Apply electrical engineering principles, including calculations of hot-spot temperatures, interpretation of DGA results, and understanding of thermal impacts on transformer insulation.
Include all intermediate calculations and reasoning steps in your analysis.
Reference specific data points from the datasets in your rationale.
Provide a brief executive summary at the beginning of your solution, highlighting your key findings and recommendations.
Present your findings in a clear, logical, and concise manner, suitable for presentation to the management team.
Identify only the transformer(s) that genuinely require immediate action, ensuring that your conclusions are based on solid evidence from the data."

</instructions>

<background>
 
Contoso, a leading energy provider, is undertaking a comprehensive analysis of its electrical distribution network to optimize maintenance planning for its critical transformers. The goal is to proactively identify transformers at risk of failure due to thermal stress, insulation degradation, overloading, and other factors, especially under projected future operating conditions. This analysis requires detailed examination of operational parameters, environmental conditions, diagnostic test results, and adherence to safety regulations.
 

Datasets
Overview
 
You are provided with detailed datasets for seven transformers (T1001 to T1007). Your task is to analyze these datasets to determine which transformer(s) require immediate maintenance or action within the next six months. The data includes:

Asset Inventory and Specifications
Load and Operational Data
Environmental Conditions
Transformer Thermal Performance Data
Insulation Deterioration Rates
Transformer Loading Guide
Technician Reports
Dissolved Gas Analysis (DGA) Results
Safety Regulations and Maintenance Constraints
Projected Future Operations
 
### Dataset 1: Asset Inventory and Specifications
 
Asset Inventory Table

| Asset ID | Location           | Install Date | Expected Lifespan (years) | Last Maintenance Date | Manufacturer       | Model     | Voltage Ratio (kV/kV) | Power Rating (MVA) | Impedance (%) | Cooling Type | Criticality Score (1-10) | Expected Retirement Date |
|----------|--------------------|--------------|---------------------------|-----------------------|--------------------|-----------|------------------------|--------------------|---------------|--------------|--------------------------|--------------------------|
| T1001    | 123 Main St        | 2005-06-15   | 25                        | 2018-05-20            | ElectroTrans Co.   | ETX-2000  | 220/110               | 100                | 8             | ONAN         | 9                        | 2030-06-15               |
| T1002    | 456 Oak Ave        | 2010-08-22   | 25                        | 2017-11-10            | PowerMax Solutions | PMX-1500  | 220/110               | 90                 | 9             | ONAF         | 7                        | 2035-08-22               |
| T1003    | 789 Industrial Rd  | 2000-03-30   | 25                        | 2015-09-05            | VoltGear Ltd.      | VG-3000   | 220/110               | 120                | 10            | ONAN         | 10                       | 2025-03-30               |
| T1004    | 321 Maple Dr       | 1995-01-12   | 25                        | 2014-04-18            | ElectroTrans Co.   | ETX-1800  | 110/33                | 80                 | 7             | ONAN         | 6                        | 2020-01-12               |
| T1005    | 654 Pine St        | 2007-12-05   | 25                        | 2016-07-25            | PowerMax Solutions | PMX-1600  | 220/110               | 109                | 6             | ONAF         | 8                        | 2032-12-05               |
| T1006    | 987 Elm St         | 1990-11-20   | 25                        | 2013-03-15            | VoltGear Ltd.      | VG-2500   | 220/110               | 90                 | 8             | ONAN         | 5                        | 2015-11-20               |
| T1007    | 222 Cedar Rd       | 2008-09-10   | 25                        | 2019-02-22            | ElectroTrans Co.   | ETX-2200  | 220/110               | 110                | 7             | ONAF         | 8                        | 2033

 												
Additional Notes:												
 												
Impedance (%) affects voltage regulation and fault levels.
Cooling Types:
ONAN: Oil Natural Air Natural
ONAF: Oil Natural Air Forced
 
### Dataset 2: Load and Operational Data
 
Monthly Average Load Data (Last 12 Months)

Transformer T1001
 

| Month         | Average Load (MVA) |
|---------------|--------------------|
| Month -11     | 80                 |
| Month -10     | 82                 |
| Month -9      | 84                 |
| Month -8      | 85                 |
| Month -7      | 87                 |
| Month -6      | 88                 |
| Month -5      | 90                 |
| Month -4      | 92                 |
| Month -3      | 90                 |
| Month -2      | 88                 |
| Month -1      | 90                 |
| Current Month | 92                 |

---	
 	
#### Transformer T1002	
 	
| Month         | Average Load (MVA) |
|---------------|--------------------|
| Month -11     | 66                 |
| Month -10     | 67                 |
| Month -9      | 68                 |
| Month -8      | 69                 |
| Month -7      | 68                 |
| Month -6      | 70                 |
| Month -5      | 71                 |
| Month -4      | 70                 |
| Month -3      | 69                 |
| Month -2      | 70                 |
| Month -1      | 69                 |
| Current Month | 70                 |

---	
 	
#### Transformer T1003	
 	
| Month         | Average Load (MVA) |
|---------------|--------------------|
| Month -11     | 110                |
| Month -10     | 112                |
| Month -9      | 114                |
| Month -8      | 116                |
| Month -7      | 116                |
| Month -6      | 116                |
| Month -5      | 116                |
| Month -4      | 116                |
| Month -3      | 118                |
| Month -2      | 115                |
| Month -1      | 116                |
| Current Month | 117                |

---	
 	
#### Transformer T1004	
 	
| Month         | Average Load (MVA) |
|---------------|--------------------|
| Month -11     | 60                 |
| Month -10     | 61                 |
| Month -9      | 62                 |
| Month -8      | 63                 |
| Month -7      | 62                 |
| Month -6      | 64                 |
| Month -5      | 65                 |
| Month -4      | 66                 |
| Month -3      | 65                 |
| Month -2      | 64                 |
| Month -1      | 65                 |
| Current Month | 65                 |

---	
 	
#### Transformer T1005	
 	
| Month         | Average Load (MVA) |
|---------------|--------------------|
| Month -11     | 90                 |
| Month -10     | 91                 |
| Month -9      | 92                 |
| Month -8      | 93                 |
| Month -7      | 94                 |
| Month -6      | 95                 |
| Month -5      | 96                 |
| Month -4      | 95                 |
| Month -3      | 96                 |
| Month -2      | 95                 |
| Month -1      | 95                 |
| Current Month | 95                 |

---	
 	
#### Transformer T1006	
 	
| Month         | Average Load (MVA) |
|---------------|--------------------|
| Month -11     | 88                 |
| Month -10     | 88                 |
| Month -9      | 88                 |
| Month -8      | 87                 |
| Month -7      | 87                 |
| Month -6      | 86                 |
| Month -5      | 86                 |
| Month -4      | 85                 |
| Month -3      | 85                 |
| Month -2      | 85                 |
| Month -1      | 85                 |
| Current Month | 85                 |

---	
 	
#### Transformer T1007	
 	
| Month         | Average Load (MVA) |
|---------------|--------------------|
| Month -11     | 95                 |
| Month -10     | 96                 |
| Month -9      | 97                 |
| Month -8      | 98                 |
| Month -7      | 99                 |
| Month -6      | 100                |
| Month -5      | 101                |
| Month -4      | 100                |
| Month -3      | 99                 |
| Month -2      | 100                |
| Month -1      | 100                |
| Current Month | 100                |

 
Projected Load Changes:
T1001: Expected to remain relatively stable with minor fluctuations due to seasonal demand variations.
T1002: Anticipated to maintain current load levels with minor fluctuations. However, a significant load increase of 15 MVA is expected in Month +4 due to the connection of a new residential development.
T1003: Expected to increase by 1% per month due to new industrial expansion starting immediately.
T1004: Expected to remain relatively stable with minor fluctuations, maintaining its current load levels.
T1006: Expected to experience a slight decrease in load by -1% per month due to energy efficiency programs implemented in the service area.
T1007: Anticipated to maintain relatively stable load levels with minor fluctuations typical of regular operational demands.
 
Notes:
The transformers T1001, T1002 (until Month +4), T1004, T1005, and T1007 are expected to have stable load profiles, with only minor fluctuations that are characteristic of normal seasonal variations and operational demands.
T1002 will experience a significant load increase in Month +4 due to the upcoming new residential development, which is projected to add 15 MVA to its current load.
The load increase for T1003 due to industrial expansion and the decrease for T1006 due to energy efficiency measures are significant and should be considered in operational planning.
All projections consider current known factors; unexpected changes in consumer behavior or unforeseen events are not accounted for in these projections.

### Dataset 3: Environmental Conditions
 
Ambient Temperature Data (Historical and Forecasted)

Ambient Temperatures (�C):

Month	Ambient Temp (�C)
Month -11	25
...	...
Current Month	40
Month +1	41
Month +2	42
Month +3	43
Month +4	42
Month +5	40
Month +6	38
 	
Notes:	
 	
Ambient temperatures peak in Month +3.
High temperatures can influence transformer cooling efficiency and insulation aging.
 

### Dataset 4: Transformer Thermal Performance Data
 
Cooling Efficiency Coefficients

| Model      | Cooling Type | CEC  | Rated TO-R (�C) | Rated HS-R (�C) |
|------------|--------------|------|-----------------|-----------------|
| VG-3000    | ONAN         | 1.0  | 55              | 30              |
| PMX-1500   | ONAF         | 0.85 | 50              | 25              |
| ETX-2000   | ONAN         | 0.95 | 55              | 30              |
| VG-2500    | ONAN         | 0.9  | 55              | 30              |
| PMX-1600   | ONAF         | 0.88 | 50              | 25              |
| ETX-1800   | ONAN         | 0.92 | 55              | 30              |
| ETX-2200   | ONAF         | 0.86 | 50              | 25              |
 				
Exponents for Temperature Calculations				
 				
ONAN Cooling:
( m = 0.8 )
( n = 1.6 )
ONAF Cooling:
( m = 0.9 )
( n = 1.8 )

Dataset 5: Insulation Deterioration Rates
 
Insulation Life Expectancy vs. Hot-Spot Temperature

| Hot-Spot Temperature (�C) | Relative Aging Rate |
|---------------------------|---------------------|
| 110                       | 1                  |
| 120                       | 2                  |
| 130                       | 4                  |
| 140                       | 8                  |
| 150                       | 16                 |

---	
 	
### Dataset 6: Transformer Loading Guide	
 	
IEEE Standard Formulas	
 	
### IEEE Standard Formulas

#### Top-Oil Temperature Rise (??_TO)
??_TO = ??_TO-R � (L / L_R)^m

#### Hot-Spot Temperature Rise (??_HS)
??_HS = ??_HS-R � (L / L_R)^n

#### Total Hot-Spot Temperature (?_HS)
?_HS = ?_Amb + ??_TO + ??_HS

#### Where:
- L: Load in MVA
- L_R: Rated Load in MVA
- m, n: Exponents based on cooling type

Dataset 7: Technician Reports
 
T1001 Report:

""No significant issues observed. Regular operational parameters.""

T1002 Report:
""Cooling fans operating efficiently. Oil color slightly darker than normal; recommend oil test.""

T1003 Report:
""Transformer oil shows higher than normal moisture content.""
""DGA indicates increased levels of ethylene and methane.""
""Cooling system operating at maximum capacity.""

T1004 Report:
""Signs of aging observed. Minor oil leaks detected.""

T1005 Report:
""External overheating signs on components.""
""Load readings higher during peak hours.""

T1006 Report:
""Transformer operating past expected lifespan.""
""Cooling fins corroded; reduced cooling efficiency expected.""

T1007 Report:
""All parameters within normal limits. No action required.""
 
Dataset 8: Dissolved Gas Analysis (DGA) Results
 
Gas Levels in Parts Per Million (ppm)

| Gas            | T1001 | T1002 | T1003 | T1004 | T1005 | T1006 | T1007 |
|----------------|-------|-------|-------|-------|-------|-------|-------|
| Hydrogen (H2)  | 20    | 40    | 150   | 30    | 50    | 80    | 25    |
| Methane (CH4)  | 10    | 25    | 120   | 15    | 30    | 60    | 12    |
| Ethylene (C2H4)| 5     | 15    | 90    | 8     | 20    | 40    | 5     |
| Ethane (C2H6)  | 4     | 10    | 60    | 6     | 15    | 35    | 4     |
| Acetylene (C2H2)| 0    | 0     | 5     | 0     | 0     | 0     | 0     |
| Carbon Monoxide (CO)| 50| 80    | 300   | 70    | 90    | 200   | 60    |
| Carbon Dioxide (CO2)| 200| 400  | 1200  | 250   | 450   | 800   | 220   |
				
Interpretation Standards:							
 							
Action Levels:
H2 > 100 ppm
CH4 > 75 ppm
C2H4 > 50 ppm
CO > 350 ppm
Presence of Acetylene (C2H2) indicates high-temperature arcing.
High levels of CO and CO2 suggest cellulose insulation degradation.
 

Dataset 9: Safety Regulations and Maintenance Constraints
 
Regulatory Guidelines:

Maintenance must not be performed when ambient temperatures exceed 40��C due to safety concerns.
Transformers operating beyond 80% of their expected lifespan require thorough inspections.
DGA action levels necessitate immediate investigation and potential maintenance.
Ambient Temperature Maintenance Window: Best scheduled during months with ambient temperatures below 40��C.

Dataset 11: SME reports

Subject: Analysis of Time-Variant Load Impact on Transformer T1005 Due to New Industrial Park Opening
 
Establishments Overview:
 

MetalWorks Manufacturing Co.
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
GreenTech Recycling Facility
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
FreshFoods Processing Plant
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
LogiTrans Distribution Center
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Quantum Data Centers Ltd.
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging
 

Time-Variant Load Profiles:
 
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):

| Time Window   | MetalWorks (MW) | GreenTech (GT) | FreshFoods (FF) | LogiTrans (LT) | Quantum Data (QD) |
|---------------|------------------|----------------|------------------|----------------|--------------------|
| 1 (00-04)     | 1.0              | 0.5            | 0.0              | 0.0            | 2.5                |
| 2 (04-08)     | 2.0              | 1.0            | 0.0              | 0.0            | 2.5                |
| 3 (08-12)     | 4.0              | 3.0            | 2.5              | 0.0            | 3.0                |
| 4 (12-16)     | 4.0              | 3.0            | 2.5              | 3.5            | 3.0                |
| 5 (16-20)     | 2.0              | 1.5            | 1.0              | 3.5            | 2.0                |
| 6 (20-24)     | 1.0              | 0.5            | 0.0              | 1.0            | 2.5                |

 					
1. Upcoming Commercial Complex Affecting Transformer T1001
 
Establishments Overview:
 
Sunrise Retail Plaza
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
EcoWise Offices
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
Harvest Foods Market
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
Swift Logistics Hub
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Nimbus Data Centers
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging
 
Time-Variant Load Profiles:
 
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):

| Time Window   | Sunrise Retail | EcoWise Offices | Harvest Foods | Swift Logistics | Nimbus Data |
|---------------|----------------|-----------------|---------------|-----------------|-------------|
| 1 (00-04)     | 0.5            | 0.3             | 0.0           | 0.0             | 0.8         |
| 2 (04-08)     | 0.8            | 0.5             | 0.0           | 0.0             | 0.8         |
| 3 (08-12)     | 1.0            | 0.6             | 0.5           | 0.0             | 0.9         |
| 4 (12-16)     | 1.0            | 0.6             | 0.5           | 0.5             | 0.9         |
| 5 (16-20)     | 0.6            | 0.5             | 0.2           | 0.5             | 0.6         |
| 6 (20-24)     | 0.5            | 0.3             | 0.0           | 0.2             | 0.8         |

 					
---

2. Residential Development Impacting Transformer T1002
 
Establishments Overview:
 
Grandview Shopping Center
Operating Voltage: 11 kV
Maximum Load: 4 MVA
Power Factor (pf): 0.85 lagging
Harmony Business Park
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.90 lagging
FreshMart Grocery
Operating Voltage: 6.6 kV
Maximum Load: 2.5 MVA
Power Factor (pf): 0.80 lagging
Express Logistics Depot
Operating Voltage: 6.6 kV
Maximum Load: 3.5 MVA
Power Factor (pf): 0.75 lagging
Apex Data Center
Operating Voltage: 11 kV
Maximum Load: 3 MVA
Power Factor (pf): 0.95 lagging
 
Time-Variant Load Profiles:
 
The daily operation is divided into six 4-hour time windows:

Window 1: 00:00 - 04:00
Window 2: 04:00 - 08:00
Window 3: 08:00 - 12:00
Window 4: 12:00 - 16:00
Window 5: 16:00 - 20:00
Window 6: 20:00 - 24:00

Load Distribution Table (Apparent Power in MVA):

| Time Window   | Grandview Mall | Harmony Business | FreshMart Grocery | Express Logistics | Apex Data Center |
|---------------|----------------|------------------|-------------------|-------------------|------------------|
| 1 (00-04)     | 0.5            | 0.3              | 0.0               | 0.0               | 0.8              |
| 2 (04-08)     | 0.8            | 0.5              | 0.0               | 0.0               | 0.8              |
| 3 (08-12)     | 1.0            | 0.6              | 0.5               | 0.0               | 0.9              |
| 4 (12-16)     | 1.0            | 0.6              | 0.5               | 0.5               | 0.9              |
| 5 (16-20)     | 0.6            | 0.5              | 0.2               | 0.5               | 0.6              |
| 6 (20-24)     | 0.5            | 0.3              | 0.0               | 0.2               | 0.8              |

		
---		

3. Small Industrial Unit Connecting to Transformer T1004
 
Overview:

A small-scale manufacturing unit is set to commence operations near Transformer T1004:

Precision Tools Ltd.: Specializing in custom tool manufacturing.
Expected Load Increase: Approximately 1.8 MVA.

4. New School Facility Supplied by Transformer T1005
 
Overview:

A new educational institution is opening in the area served by Transformer T1005:

Sunrise Elementary School: Modern facilities with standard energy consumption.
Expected Load Increase: Approximately 1.2 MVA.

5. Community Center Addition Affecting Transformer T1007
 
Overview:

Construction of a community center is nearing completion in the vicinity of Transformer T1007:

Riverdale Community Hall: Hosting events and activities for local residents.
Expected Load Increase: Approximately 1 MVA.


6. Small Business Cluster Connecting to Transformer T1003
 
Overview:

A cluster of small businesses is opening near Transformer T1003:

Shops Include:
Cafe Delight: Boutique coffee shop.
Health Rx Pharmacy: Local pharmacy outlet.
Bloom Floral Shop: Florist with refrigeration needs.
TechFix IT Services: Electronics repair shop.
FitLife Gym Studio: Fitness center with standard equipment.
Expected Load Increase: Approximately 2 MVA combined.
 

</background>
"""

In [46]:
resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 5843 | Completion tokens = 8816 | Total tokens = 14659
Total cost = 0.043038006344 EUR
Completed in 00:01:52

Below is a brief executive summary followed by an in‐depth, step‐by‐step analysis that shows how data from thermal performance, load projections, DGA test results, technician observations, and asset/lifespan information lead us to prioritize transformer T1003—and also raise a red flag on T1006—for immediate action within the next six months.

────────────────────────────
Executive Summary:
• Transformer T1003 (VG-3000) is operating with very high hot‐spot temperatures (≈123°C under current conditions) and shows alarmingly high dissolved gas levels (H₂ = 150 ppm, CH₄ = 120 ppm, C₂H₄ = 90 ppm) that exceed all action thresholds. In combination with reports of excess oil moisture and a cooling system already working at capacity—and with monthly load increases from industrial expansion—T1003 is at high risk of insulation degradation and imminent failure.
• Transforme

## Customer retention

In [47]:
prompt = """
<title>
Customer Retention Analysis at Contoso: Uncovering the Root Cause of Subscription Cancellation
</title>

<background>

Contoso is a leading fashion and clothing retailer that offers a premium subscription service called **Contoso Plus**. The service includes benefits like free shipping, early access to new collections, and personalized styling recommendations. Recently, a long-term customer, **Emily Thompson**, emailed customer support expressing her desire to cancel her **Contoso Plus** subscription, citing the cost as the main issue.

As a customer support analyst, you have access to various datasets to investigate the underlying reasons for Emily's cancellation request. Your goal is to identify the true cause of her dissatisfaction and propose solutions to retain her as a valued customer.

Below are the datasets available for your analysis:

---

### Dataset 1: Customer Profile

This dataset provides an overview of Emily's interaction with Contoso.

| Customer ID | Name            | Age | Membership Duration (Months) | Total Spend ($) | Average Monthly Spend ($) | Preferred Categories    |
|-------------|-----------------|-----|------------------------------|-----------------|---------------------------|-------------------------|
| C1024       | Emily Thompson  | 35  | 24                           | 4,800           | 200                       | Dresses, Shoes, Accessories |

---

### Dataset 2: Purchase History

A detailed list of Emily's purchases over the past 6 months.

| Order ID   | Date       | Items Purchased                    | Total Amount ($) | Discount Applied (%) | Date Delivered | Return Flag |
|------------|------------|------------------------------------|------------------|----------------------|-----------------------|-------------|
| O5678      | 2023-03-15 | Summer Floral Dress, Sun Hat       | 150              | 10                   | 2023-03-19                     | No          |
| O5721      | 2023-04-10 | Leather Ankle Boots                | 120              | 15                   | 2023-04-13                     | No          |
| O5789      | 2023-05-05 | Silk Scarf                         | 80               | 0                    | 2023-05-25                     | Yes         |
| O5832      | 2023-06-18 | Casual Sneakers                    | 90               | 5                    | 2023-06-21                     | No          |
| O5890      | 2023-07-22 | Evening Gown, Clutch Bag           | 300              | 20                   | 2023-08-05                | No          |
| O5935      | 2023-08-30 | Denim Jacket                       | 110              | 0                    | 2023-09-03                     | Yes         |
| O5970      | 2023-09-12 | Fitness Leggings, Sports Bra       | 130              | 25                   | 2023-09-18                     | No          |

---

### Dataset 3: Customer Service Interactions

Transcripts of Emily's past interactions with customer support.

#### Interaction 1: Live Chat on 2023-06-20

**Agent:** Hello Emily, how can I assist you today?

**Emily:** Hi, I just received my order O5789, and wanted to swap it for another colour

**Agent:** Sure, that's fine- feel free to send it back or change it in store.

**Emily:** Ok, I'll just send it back then

**Agent:** Certainly. I've initiated the return process. You'll receive an email with the return instructions.

**Emily:** Thank you.

---

#### Interaction 2: Phone Call on 2023-07-25

**Agent:** Good afternoon, this is Contoso customer service. How may I help you?

**Emily:** I'm calling about my order O5890. I need the gown for an event this weekend, and just want to make sure it will be delivered on time as it's really important.

**Agent:** Let me check... it seems like the delivery is on track. It should be there on time.

**Emily:** Ok thanks.

---

#### Interaction 3: Email on 2023-09-15

*Subject:* Membership Cancellation Request

*Body:*

Hello,

I want to cancel my Contoso Plus subscription. The cost is becoming too high for me.

- Emily Thompson

---

### Dataset 4: Website Activity Log *DS*

Logs of Emily's activity on the Contoso website.

| Date       | Pages Visited                                  | Time Spent (Minutes) |
|------------|------------------------------------------------|----------------------|
| 2023-09-10 | Homepage, New Arrivals, Dresses                | 15                   |
| 2023-09-11 | Account Settings, Subscription Details         | 5                    |
| 2023-09-12 | FAQ, Return Policy                             | 3                    |
| 2023-09-13 | Careers Page, Company Mission                  | 2                    |
| 2023-09-14 | Sale Items, Accessories                        | 10                   |

---

### Dataset 5: Delivery Performance Metrics

Company-wide delivery statistics over the past 6 months.

| Month      | Average Delivery Time (Days) | On-Time Delivery Rate (%) | Customer Complaints Received |
|------------|------------------------------|---------------------------|------------------------------|
| March      | 3                            | 98                        | 15                           |
| April      | 4                            | 95                        | 20                           |
| May        | 5                            | 92                        | 30                           |
| June       | 6                            | 88                        | 50                           |
| July       | 7                            | 85                        | 70                           |
| August     | 4                            | 94                        | 25                           |
| September  | 3                            | 97                        | 10                           |

---

### Dataset 6: Product Return Rates *DS*

Return rates for different product categories.

| Category      | Return Rate (%) |
|---------------|-----------------|
| Dresses       | 15              |
| Shoes         | 10              |
| Accessories   | 8               |
| Outerwear     | 12              |
| Sportswear    | 9               |

---

### Dataset 7: Social Media Sentiment Analysis *DS*

Aggregated customer sentiment towards Contoso.

| Month      | Positive Mentions | Negative Mentions | Neutral Mentions |
|------------|-------------------|-------------------|------------------|
| March      | 500               | 50                | 200              |
| April      | 480               | 60                | 220              |
| May        | 450               | 80                | 250              |
| June       | 400               | 120               | 300              |
| July       | 350               | 150               | 320              |
| August     | 480               | 70                | 230              |
| September  | 510               | 40                | 210              |

---

### Dataset 8: Store Visit History

Records of Emily's visits to physical Contoso stores.

| Date       | Store Location        | Purpose           | Outcome                                 |
|------------|-----------------------|-------------------|-----------------------------------------|
| 2023-05-12 | Downtown Outlet       | Browsing          | Purchased a Silk Scarf (O5789)          |
| 2023-07-20 | Uptown Mall           | Personal Styling  | Booked a session but didn't attend |
| 2023-08-05 | Midtown Boutique      | Browsing          | No purchase                             |

---

### Dataset 9: Subscription Benefits Utilization

Emily's usage of Contoso Plus benefits.

| Benefit                       | Usage Frequency (Last 6 Months) |
|-------------------------------|---------------------------------|
| Free Shipping                 | 7                               |
| Early Access to Collections   | 2                               |
| Exclusive Discounts           | 1                               |
| Personalized Styling Sessions | 0                               |

---

### Dataset 10: Customer Feedback Surveys

Emily's responses to post-purchase surveys.

#### Survey After Order O5678 (2023-03-16)

- **Satisfaction Rating (1-5):** 5
- **Comments:** ""Loved the summer dress! Fast delivery.""

#### Survey After Order O5970 (2023-09-13)

- **Satisfaction Rating (1-5):** 4
- **Comments:** ""Happy with the sportswear. Quick delivery.""

---

### Dataset 11: Email Marketing Engagement *DS*

Emily's engagement with promotional emails.

| Campaign                  | Opened | Clicked | Unsubscribed |
|---------------------------|--------|---------|--------------|
| Summer Sale               | Yes    | Yes     | No           |
| New Arrivals              | Yes    | No      | No           |
| Exclusive Member Offers   | No     | No      | No           |
| Personal Styling Invite   | No     | No      | No           |
| Autumn Collection Preview | Yes    | Yes     | No           |

---

### Dataset 12: Customer Churn Analysis *DS*

Metrics related to subscription cancellations across all customers.

| Reason for Cancellation          | Percentage (%) |
|----------------------------------|----------------|
| Service Dissatisfaction          | 40             |
| Financial Reasons                | **3**          |
| Competitor Offer                 | 15             |
| Moving to a Non-Service Area     | 5              |
| Other                            | 37             |

---

### Dataset 13: Warehouse Incident Reports

Reports of operational issues affecting order fulfillment.

| Date       | Incident Description                              | Affected Orders |
|------------|---------------------------------------------------|-----------------|
| 2023-06-15 | Inventory system outage                           | 100             |
| 2023-07-18 | Logistics partner strike                          | 250             |
| 2023-08-25 | Warehouse flooding due to heavy rain              | 150             |

---

### Dataset 14: Competitor Pricing Analysis *DS*

Comparison of product pricing with major competitors.

| Product Category | Contoso Average Price ($) | Competitor Average Price ($) |
|------------------|------------------------------|------------------------------|
| Dresses          | 120                          | **100**                      |
| Shoes            | 100                          | 105                          |
| Accessories      | 60                           | 55                           |
| Sportswear       | 80                           | 85                           |

---

### Dataset 15: Loyalty Program Overview

Details of Contoso's loyalty point accrual and redemption for Emily.

| Total Points Earned | Points Redeemed | Current Point Balance | Points Expiring Next Month |
|---------------------|-----------------|-----------------------|----------------------------|
| 4,800               | 3,600           | 1,200                 | 1,200                      |

---

### Dataset 16: Unauthorized Access Attempts *DS*

Security logs related to Emily's account.

| Date       | IP Address     | Location       | Successful Login? |
|------------|----------------|----------------|-------------------|
| 2023-06-20 | 192.168.1.1    | Home Network   | Yes               |
| 2023-07-22 | 203.0.113.45   | Unknown        | No                |
| 2023-08-15 | 198.51.100.23  | Office Network | Yes               |

---

</background>

<question>

### Task

As a customer support analyst at Contoso, you are tasked with:

1. **Identifying the primary reason for Emily Thompson's cancellation request**.
2. **Providing a detailed analysis** of the factors contributing to her dissatisfaction.
3. **Drafting a personalized email** responding to Emily's concerns, acknowledging the specific issues that have been frustrating for her, and offering solutions to retain her as a premium subscriber.

Your analysis should:
- Synthesize information from multiple datasets provided.
- Highlight key incidents that have impacted Emily's experience.
- Quantify any patterns or trends relevant to her case.

</question>

<instruction>

- Carefully review all the datasets to extract relevant information.
- Include all intermediate analyses and reference specific data points from the datasets to support your conclusions.
- Provide a brief **executive summary** at the beginning of your solution, summarizing your key findings and recommendations.
- Draft a professional and empathetic email to Emily, addressing her concerns.
- Present your findings in a clear and structured manner, suitable for presentation to senior management.
- Use markdown formatting for headings, tables, and any figures.

</instruction>"
"""

In [48]:
resp = o3mini(prompt, reasoning_effort='high')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 2602 | Completion tokens = 5515 | Total tokens = 8117
Total cost = 0.02582047426 EUR
Completed in 00:01:08

Below is the complete analysis—with an executive summary, detailed discussion that references multiple datasets—and a draft email to Emily Thompson.

--------------------------------------------------

# Executive Summary

Our investigation indicates that while Emily’s email states that "the cost is becoming too high," the underlying issue is a perceived lack of value from her Contoso Plus membership. Despite her long-term relationship and healthy spend (Dataset 1), Emily’s usage of premium benefits is very limited (Dataset 9 and Dataset 11). In addition, her order history shows occasional frustrations (e.g., longer delivery times and returned orders in Dataset 2, coupled with external operational issues in Datasets 5 and 13) that may have further diminished her overall experience. Notably, company-wide churn data (Dataset 12) shows that only 3% of cancellations a

## Python coding

In [49]:
prompt = """
Créez une application GUI Python avec différentes formes 3D rotatives. L'interface doit avoir un thème mode nuit élégant et moderne avec un fond sombre et des couleurs contrastées de type néon. Sur le côté gauche de la fenêtre, implémentez un panneau de navigation à onglets qui permet aux utilisateurs de sélectionner différentes formes 3D (comme un cube, une sphère, une pyramide et un tore). Lorsqu'une forme est sélectionnée, elle doit apparaître dans la zone d'affichage principale et tourner continuellement dans l'espace 3D. Utilisez OpenGL ou une bibliothèque graphique similaire pour gérer le rendu 3D. L'application doit avoir des animations fluides, une interface utilisateur intuitive et une option pour ajuster la vitesse de rotation. Assurez-vous qu'elle est multiplateforme et fonctionne efficacement.
"""

resp = o3mini(prompt, reasoning_effort='medium')
print(f"\033[1;34m{resp.choices[0].message.content}\033[0m")

Prompt Tokens = 179 | Completion tokens = 4518 | Total tokens = 4697
Total cost = 0.019108272912000002 EUR
Completed in 00:00:56

Ci-dessous un exemple complet de code Python qui utilise PyQt5 et PyOpenGL pour créer une application GUI au thème « mode nuit » avec un panneau de navigation à onglets sur la gauche permettant de sélectionner et de visualiser des formes 3D animées (cube, sphère, pyramide et tore). Vous devez avoir installé PyQt5 et PyOpenGL (pip install PyQt5 PyOpenGL) pour l’exécuter. Le code ci‐contre peut être amélioré et enrichi, mais il offre une base fonctionnelle et multiplateforme :

-----------------------------------------------------------
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Application GUI en mode nuit avec affichage de formes 3D animées.
Utilisation de PyQt5 pour l'interface et PyOpenGL pour le rendu.
"""

import sys, math
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QHBoxLayout, QVBoxLayout,
                             QTabW

## Webapp

In [64]:
def o3_gradio_fn(prompt, reasoning_effort='low'):
    """
    Generates a response from the o3mini model based on the given prompt and reasoning effort, 
    computes the cost of the tokens used, and returns the response content along with a message 
    displaying token usage and total cost.

    Args:
        prompt (str): The input prompt for the model.
        reasoning_effort (str, optional): The level of reasoning effort for the model. Defaults to 'low'.
    
    Returns:
        tuple: A tuple containing the response content (str) and a message (str) displaying token usage, 
               total cost, and elapsed time.
    """
    start = time.time()
    # Calling the model
    resp = o3mini(prompt, reasoning_effort=reasoning_effort)
    elapsed = time.time() - start
    # Cost computing
    cost_input_token = 1.0470 / 1_000_000
    cost_output_token = 4.187884 / 1_000_000
    total_cost = (resp.usage.prompt_tokens * cost_input_token) + (resp.usage.completion_tokens * cost_output_token)
    # Elapsed time
    elapsed_time_msg = f"Completed in {time.strftime('%H:%M:%S' + str(elapsed % 1)[2:2], time.gmtime(elapsed))}"
    # Message generation
    msg_to_display = f"Prompt Tokens = {resp.usage.prompt_tokens} | Completion tokens = {resp.usage.completion_tokens} | Total tokens = {resp.usage.total_tokens}\nTotal cost = {total_cost} EUR\n{elapsed_time_msg}"

    return resp.choices[0].message.content, msg_to_display

In [65]:
def download_file(response):
    """
    Downloads the given response content into a temporary file with a timestamped filename.

    Args:
        response (str): The content to be written into the file.
    
    Returns:
        str: The path to the temporary file containing the response content.
    """
    now = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    with tempfile.NamedTemporaryFile(delete=False, prefix=now,
                                     suffix=".txt") as tmp_file:
        tmp_file.write(response.encode("utf-8"))
        tmp_file_path = tmp_file.name

    return tmp_file_path

In [66]:
webapp_css = """
.blue-text textarea {
    color: blue !important;
    font-weight: bold;
}
.green-text textarea {
    color: green !important;
    font-weight: bold;
}
.purple-text textarea {
    color: purple !important;
}
"""

with gr.Blocks(theme="default", css=webapp_css) as webapp:
    gr.Markdown("# Azure AI Foundry o3-mini")
    gr.Image("o3.jpg", width=400)
    
    prompt_input = gr.Textbox(label="➡️ Enter your prompt",
                              lines=3,
                              show_copy_button=True,
                              elem_classes=["purple-text"])
    
    reasoning_radio = gr.Radio(choices=["high", "medium", "low"],
                               label="💡 Reasoning Effort",
                               value="low")
    
    submit_button = gr.Button("✅ Running the o3-mini model")
    
    output_text = gr.Textbox(label="Response",
                             show_copy_button=True,
                             elem_classes=["blue-text"])
    
    msg_box = gr.Textbox(label="Token Usage and Cost",
                         interactive=False,
                         show_copy_button=True,
                         elem_classes=["green-text"])
    with gr.Row():
        download_button = gr.Button("🏷️ Download results into a file")

    submit_button.click(o3_gradio_fn,
                        inputs=[prompt_input, reasoning_radio],
                        outputs=[output_text, msg_box])
    download_button.click(download_file, inputs=output_text, outputs=gr.File())

    gr.Markdown("🚀 Powered by Azure AI Foundry")


webapp.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://3083b55b79f23e5cd5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Prompt Tokens = 10 | Completion tokens = 117 | Total tokens = 127
Total cost = 0.000500452428 EUR
Completed in 00:00:02

